In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Modelagem

# Pontos



In [3]:
!pip3 install tpot

     |████████████████████████████████| 87 kB 3.1 MB/s 
     |████████████████████████████████| 160 kB 40.2 MB/s 
     |████████████████████████████████| 173.6 MB 7.9 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11956 sha256=34bfc833ff2f4189d39cee5269a82eeecda80efde0389385f106aad544526a23
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [4]:
import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split 
#import autosklearn as ask
import tpot

In [5]:
my_df = joblib.load('/content/drive/MyDrive/Colab Notebooks/Previsão de pessoas com Diabetes/data/Modelagem.gz')
my_df.head()

,diabetes,pressao_alta,colesterol_alto,testou_colesterol,IMC,fumante,AVC,ataque_cardiaco,ativ_fisica,consome_frutas,...,seguro_saude,n_foi_med_custo,auto_aval_saude,saude_mental,saude_fisica,dif_escadas,sexo,class_idade,class_educ,class_renda
0,0,1,1,1,40,1,0,0,0,0,...,1,0,5,18,15,1,0,9,4,3
1,0,0,0,0,25,1,0,0,1,0,...,0,1,3,0,0,0,0,7,6,1
2,0,1,1,1,28,0,0,0,0,1,...,1,1,5,30,30,1,0,9,4,8
3,0,1,0,1,27,0,0,0,1,1,...,1,0,2,0,0,0,0,11,3,6
4,0,1,1,1,24,0,0,0,1,1,...,1,0,2,3,0,0,0,11,5,4


In [6]:
my_df.columns

Index(['diabetes', 'pressao_alta', 'colesterol_alto', 'testou_colesterol',
       'IMC', 'fumante', 'AVC', 'ataque_cardiaco', 'ativ_fisica',
       'consome_frutas', 'consome_vegetais', 'consumo_alcool', 'seguro_saude',
       'n_foi_med_custo', 'auto_aval_saude', 'saude_mental', 'saude_fisica',
       'dif_escadas', 'sexo', 'class_idade', 'class_educ', 'class_renda'],
      dtype='object')

In [7]:
#my_df = my_df[[ 'class_renda','diabetes',]]

In [8]:
features = my_df.drop(['diabetes'],1)
label = my_df['diabetes']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [9]:
features.columns

Index(['pressao_alta', 'colesterol_alto', 'testou_colesterol', 'IMC',
       'fumante', 'AVC', 'ataque_cardiaco', 'ativ_fisica', 'consome_frutas',
       'consome_vegetais', 'consumo_alcool', 'seguro_saude', 'n_foi_med_custo',
       'auto_aval_saude', 'saude_mental', 'saude_fisica', 'dif_escadas',
       'sexo', 'class_idade', 'class_educ', 'class_renda'],
      dtype='object')

## Seleção de Features

A idéia é classificar as features quanto o nível de relevância em relação ao atributo alvo, no caso "diabetes".

In [10]:
features_list = ('pressao_alta', 'colesterol_alto', 'testou_colesterol', 'IMC',
                 'fumante', 'AVC', 'ataque_cardiaco', 'ativ_fisica', 'consome_frutas',
                 'consome_vegetais', 'consumo_alcool', 'seguro_saude', 'n_foi_med_custo',
                 'auto_aval_saude', 'saude_mental', 'saude_fisica', 'dif_escadas',
                 'sexo', 'class_idade', 'class_educ', 'class_renda')

k_best_features = SelectKBest(k='all')
k_best_features.fit_transform(features, label)
k_best_features_scores = k_best_features.scores_
raw_pairs = zip(features_list[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

k_best_features_final = dict(ordered_pairs[:15])
best_features = k_best_features_final.keys()
print ("Melhores features:")
k_best_features_final

Melhores features:


{'AVC': 1003.7093491808675,
 'IMC': 1174.2327653063705,
 'ataque_cardiaco': 2817.079573322745,
 'ativ_fisica': 8198.136218536154,
 'class_educ': 9079.734632702635,
 'class_renda': 4484.838370760624,
 'colesterol_alto': 20000.580081476644,
 'consome_frutas': 3794.1054918002096,
 'consumo_alcool': 892.7576746839748,
 'dif_escadas': 8009.451199160566,
 'fumante': 13382.153420743054,
 'saude_fisica': 1433.8909054794103,
 'saude_mental': 25233.520775754532,
 'sexo': 13169.679188206317,
 'testou_colesterol': 11737.119165380496}

Separação da base em treino e teste, utilizando 80% pra treino e 20% pra teste.

In [11]:
#features = my_df.loc[:,['AVC', 'IMC', 'ataque_cardiaco','ativ_fisica', 'class_educ', 'class_renda', 'colesterol_alto', 'consome_frutas']]
X_train, X_test, y_train, y_test = train_test_split(features, label, train_size=0.8)

## AutoML

O primeiro teste é com a lib autosklearn

In [12]:
#model = ask.classification.AutoSklearnClassifier(ensemble_size=10, #size of the end ensemble (minimum is 1)
#                                                 time_left_for_this_task=120, #the number of seconds the process runs for
#                                                 per_run_time_limit=30) #maximum seconds allocated per model

#model.fit(X_train, y_train) #begin fitting the search model
#print(model.sprint_statistics()) #print statistics for the search
#y_predictions = model.predict(X_test) #get predictions from the model

O segundo teste é com a lib tpot

In [ ]:
pipeline_optimizer = tpot.TPOTClassifier(generations=10, #number of iterations to run the training
                                         population_size=40, #number of individuals to train
                                         cv=10) #number of folds in StratifiedKFold
pipeline_optimizer.fit(X_train, y_train) #fit the pipeline optimizer - can take a long time
print(pipeline_optimizer.score(X_test, y_test)) #print scoring for the pipeline
pipeline_optimizer.export('/content/drive/MyDrive/Colab Notebooks/tpot_exported_pipeline.py') #export the pipeline - in Python code!